# Set up for model experiments

We'll do the following here:

- Create distinct catalogs
- Document parameters changes that will accompany each
- Create yamls for each experiment

# Catalogs

In [91]:
import pandas as pd
from pathlib import Path
import yaml
import os
import re
import glob 
import geopandas as gpd
import shutil

In [11]:
catalog = pd.read_csv("../data/ftw-mappingafrica-combined-catalog.csv")
catalog.query("dataset == 'ftw'").to_csv("../data/ftw-catalog.csv", index=False)

## Single Time Point

#### Combined catalogs

Including small test catalogs 

Update: catalog will be adjusted to drop Portugal and presence-only data from validation set for FTW

In [ ]:
catalog = pd.read_csv("../data/ftw-mappingafrica-combined-catalog.csv")
# catalog["split"]

# 22 records in India had no split assigned--placed them in train
catalog["split"] = catalog["split"].replace("none", "train")

catalog['split'] = catalog['split'].astype(str).str.strip().str.lower()
catalog['country'] = catalog['country'].astype(str).str.strip().str.lower()
catalog['null_prop'] = pd.to_numeric(catalog['null_prop'], errors='coerce')  # NaN if non-numeric

mask = ~(
    catalog['split'].isin(['validate', 'test'])
    & (
        (catalog['null_prop'] > 0) | (catalog['country'] == 'portugal')
    )
)
catalogr = catalog[mask].copy().reset_index(drop=True)

# FTW and full catalogs dropping Portugal and presence-only data from val/test
(catalogr.query("dataset == 'ftw'")
 .to_csv("../data/ftw-catalog2.csv", index=False))
(catalogr
 .to_csv("../data/ftw-mappingafrica-combined-catalog2.csv", index=False))

# small FTW for rapid model testing
ftw_small = (
    catalogr
     .query("dataset == 'ftw'")
     .sample(10000, random_state=42)
).reset_index(drop=True)#.to_csv("../data/ftw-catalog-small.csv", index=False))
# ftw_small["split"].value_counts()
ftw_small.to_csv("../data/ftw-catalog-small.csv", index=False)


Make another small catalog for debugging, including null prop > 0 in validation data

In [ ]:
ftw_catalog = pd.read_csv("../data/ftw-catalog.csv")
ftw_small_debug = (ftw_catalog
                   .sample(1000, random_state=42)
                   .reset_index(drop=True))
# ftw_small_debug.query("split == 'validate' & null_prop > 0")
ftw_small_debug.to_csv("../data/ftw-catalog-small-debug.csv", index=False)

#### Mapping Africa

In [ ]:
catalog = pd.read_csv("../data/ftw-mappingafrica-combined-catalog.csv")
(catalog.query("dataset == 'mappingafrica'")
 .to_csv("../data/mappingafrica-catalog.csv", index=False))

Mapping Africa countries with >30 validation samples

In [ ]:
ma_catalog = pd.read_csv("../data/mappingafrica-catalog.csv")
ma_countries = (ma_catalog.query("split == 'validate'")
 .country
 .value_counts()
 .reset_index()
 .query("count > 30")
 .sort_values("country")
 .drop("count", axis=1)
 )
with open("../scripts/ma-countries.txt", "w") as f:
    for c in ma_countries.country.tolist():
        f.write(f"{c}\n")

#### Inference catalogs 

For inference testing, both chips and tiles

In [ ]:

# # Recursively find all files in /Users/lestes/images/tiles and subfolders
tiles_list = [
    os.path.relpath(f, "/Users/lestes/images/tiles")
    for f in glob.glob("/Users/lestes/images/tiles/**/*", recursive=True)
    if os.path.isfile(f) and f.endswith("cog.tif")
    # (os.path.basename(f).startswith(("tile910", "tile513", "tile765"))      
]

# tile numbers of interest
tile_ids = ["910217", "910218", "513706", "513726", "765503"]  

# Filter tiles_list for specified tile_ids and keep only the last (latest year) 
# For each tile_id, keep only the image from the most recent year (folder)
tile_dict = {}
for f in tiles_list:
  basename = os.path.basename(f)
  for tid in tile_ids:
    if basename.startswith(f"tile{tid}"):
      year_folder = os.path.dirname(f)
      try:
        year_int = int(year_folder)
      except ValueError:
        continue  # skip if folder is not a year
      # Keep the file if it's the first seen or has a greater year
      if tid not in tile_dict or year_int > int(os.path.dirname(tile_dict[tid])):
        tile_dict[tid] = f
tiles_selected = list(tile_dict.values())
                 
tile_catalog = pd.DataFrame(
    [{"name": re.sub("tile", "", os.path.basename(f).split("_")[0]),  
      # "year": os.path.dirname(f), 
      "date": f"{os.path.basename(f).split("_")[1]}-15",
      "window_b": f}
      for f in tiles_selected]
).sort_values(by=["name", "date"]).reset_index(drop=True)
# tile_catalog.to_csv("../data/mappingafrica-tile-catalog.csv", index=False)
# (tile_catalog.query("name in ['910217', '910218'] & year == '2024'")
#  .to_csv("../data/mappingafrica-tile-catalog-small.csv", index=False))

tile_catalog


,name,date,window_b
0,513706,2023-11-15,2023/tile513706_2023-11_buf179_cog.tif
1,513726,2023-11-15,2023/tile513726_2023-11_buf179_cog.tif
2,765503,2022-03-15,2022/tile765503_2022-03_buf179_cog.tif
3,910217,2024-06-15,2024/tile910217_2024-06_buf179_cog.tif
4,910218,2024-06-15,2024/tile910218_2024-06_buf179_cog.tif


In [8]:
pth = "~/Dropbox/projects/activelearning/mappingafrica/campaigns/data/grids"
tiles = [gpd.read_file(f"{pth}/{cntry}_tiles.geojson")[["tile", "geometry"]] 
         for cntry in ["ghana", "congo", "zambia"]]
tiles = pd.concat(tiles, axis=0)
tiles["tile"] = tiles["tile"].astype(int).astype(str)
tiles = tiles.query("tile in @tile_ids").reset_index(drop=True)

tiles_gdf = (
    pd.merge(tile_catalog, tiles, left_on="name", right_on="tile", how="inner")
    .drop(columns=["tile"])
    .pipe(gpd.GeoDataFrame, geometry="geometry", crs="EPSG:4326")
)
tiles_gdf.to_file("../data/mappingafrica-tile-catalog-small.geojson", 
                  driver="GeoJSON")


#### FTW long

Make catalog placing FTW win_a labels under win_b to let model learn from both time points. Make new combined catalog. 



In [18]:
ftw_catalog = catalog.query("dataset == 'ftw'")
ftw_catalog2 = ftw_catalog.copy()
ftw_catalog2["window_b"] = ftw_catalog2["window_a"]
ftw_catalog_long = (pd.concat([ftw_catalog, ftw_catalog2], axis=0)
                    .reset_index(drop=True))
ftw_catalog_long["window_a"] = ""
# .to_csv("../data/ftw-mappingafrica-combined-catalog.csv", index=False)

# Long FTW catalog
ftw_catalog_long.to_csv("../data/ftw-catalog-long.csv", index=False)

# Long combined
catalog_long = pd.concat([ftw_catalog_long, 
                          catalog.query("dataset == 'mappingafrica'")], axis=0)
# list(catalog_long.shape)
# catalog_long.head()
catalog_long.to_csv("../data/ftw-mappingafrica-combined-catalog-long.csv", 
                    index=False)

And new catalog that drops from validation set all presence-only data, but keeping Portugal this time in one (long2), and for consistency with previous dropping Portugal and presence only in the others (long3). 

In [ ]:
mask = ~(
    catalog_long['split'].isin(['validate', 'test'])
    & (catalog_long['null_prop'] > 0) 
)

catalogr = catalog_long[mask].copy().reset_index(drop=True)
print(len(catalogr), len(catalog_long))

(catalogr
 .to_csv("../data/ftw-mappingafrica-combined-catalog-long2.csv", index=False))

(catalogr.query("dataset == 'ftw'")
 .to_csv("../data/ftw-catalog-long2.csv", index=False))

mask = ~(
    catalog_long['split'].isin(['validate', 'test'])
    & (
        (catalog_long['null_prop'] > 0) | 
        (catalog_long['country'] == 'portugal')
    )
)

catalogr = catalog_long[mask].copy().reset_index(drop=True)
print(len(catalogr), len(catalog_long))

(catalogr
 .to_csv("../data/ftw-mappingafrica-combined-catalog-long3.csv", index=False))

(catalogr.query("dataset == 'ftw'")
 .to_csv("../data/ftw-catalog-long3.csv", index=False))

catalogr

FTW country list for validation metrics

In [47]:
ftw_catalog_long = pd.read_csv(
    "../data/ftw-catalog-long3.csv"
)
ftw_countries = (ftw_catalog_long.query("split == 'validate'")
 .country
 .value_counts()
 .reset_index()
 .sort_values("country")
 .drop("count", axis=1)
 )
with open("../scripts/ftw-countries.txt", "w") as f:
    for c in ftw_countries.country.tolist():
        f.write(f"{c}\n")


#### Mapping Africa and Full Catalogs with S2

Setting up the catalog requires some gymnastics because there are 7 sites where labels are collected from different dates, and two separate acquisition catalogs with dates were made. These had to be combined to have the correct image line up with each duplicated label before combining with the current Planet catalog. 

In [ ]:
# full catalog, missing 7 records for duplicates
s2_winb_catalog = pd.read_csv("../external/inputs/primary_merged_dedup.csv")
s2_winb_catalog.rename(columns={"chip_id": "name"}, inplace=True)

s2_winb_catalog_missing7 = pd.read_csv(
    "../external/inputs/dupe_chip_id_7_other_years.csv"
)
s2_winb_catalog_missing7.rename(columns={"chip_id": "name"}, inplace=True)
s2_winb_catalog_missing7.drop(s2_winb_catalog_missing7.columns[0:2], axis=1, 
                              inplace=True)
# s2_winb_catalog_missing7

s2_dir = "/Users/LEstes/data/labels/cropland/mappingafrica-s2-256/images"

fns = [f for f in os.listdir(s2_dir)]
fns = pd.DataFrame(fns).rename(columns ={0: "image"})
fns["name"] = fns["image"].apply(lambda x: x.split("_")[0])
fns["nmlen"] = fns["image"].apply(lambda x: len(x.split("_")))
dup_names = s2_winb_catalog_missing7['name'].to_list()

# file names matching images at duplicate sites w/out year embedded in filename
s2_winb_catalog_missing7fn = pd.merge(
    s2_winb_catalog_missing7, 
    fns.query("name.isin(@dup_names) & nmlen == 3"),
    on="name"
)
# w/ year embedded in filename
s2_winb_catalog_nomissing7fn = pd.merge(
    s2_winb_catalog.query("name.isin(@dup_names)"),
    fns.query("name.isin(@dup_names) & nmlen == 4"),
    on="name"
)

# filenames for non-duplicated catalog
s2_winb_catalog_nodups = pd.merge(
    s2_winb_catalog.query("~name.isin(@dup_names)"), 
    fns.query("~name.isin(@dup_names)")
)

s2_winb_catalog = pd.concat([
    s2_winb_catalog_nodups, 
    s2_winb_catalog_nomissing7fn, 
    s2_winb_catalog_missing7fn
], ignore_index=True)

cnames = ['name', 'country', 'planet_date', 's2_primary_date', "image"]       
s2_winb_catalog = s2_winb_catalog[cnames]

s2_winb_catalog.rename(columns={"chip_id": "name", "s2_primary_date": "date"}, 
                       inplace=True)
s2_winb_catalog.sort_values("name", inplace=True)

s2_winb_catalog["date"] = pd.to_datetime(
    s2_winb_catalog["date"], utc=True, errors="coerce"
).dt.strftime("%Y-%m-%d")
s2_winb_catalog["pl_date"] = pd.to_datetime(
    s2_winb_catalog["planet_date"], utc=True, errors="coerce"
).dt.strftime("%Y-%m-%d")

s2_winb_catalog["year"] = (s2_winb_catalog["pl_date"]
                           .apply(lambda x: x.split("-")[0]))

cnames = ['name', 'country', 'date', 'pl_date', "year", "image"]       
s2_winb_catalog = s2_winb_catalog[cnames]

# add file name
s2_winb_catalog["window_b"] = s2_winb_catalog.apply(
    lambda x: f"mappingafrica-s2-256/window_b/{x['name']}_S2_"\
        f"{'-'.join(x['date'].split('-')[:2])}.tif", axis=1 
)

# s2_winb_catalog.isna().any()
# change unknown countries to first two of name, except Namibia
mask = s2_winb_catalog["country"] == "UNK"
s2_winb_catalog.loc[mask, "country"] = s2_winb_catalog.loc[mask, "name"].str[:2]

mask2 = s2_winb_catalog["country"] == "NAN"
s2_winb_catalog.loc[mask2, "country"] = "NAM"

# check correct assignment in duplicated part of catalog
s2_winb_catalog.query("name.isin(@dup_names)")
# s2_winb_catalog.query("country=='NAM'")

,name,country,date,pl_date,year,image,window_b
39069,GH0185923,GH,2018-12-03,2018-12-01,2018,GH0185923_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/GH0185923_S2_201...
39062,GH0185923,GH,2021-04-01,2021-04-01,2021,GH0185923_2021_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/GH0185923_S2_202...
39070,GH0238214,GH,2018-12-03,2018-12-01,2018,GH0238214_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/GH0238214_S2_201...
39063,GH0238214,GH,2021-12-02,2021-12-01,2021,GH0238214_2021_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/GH0238214_S2_202...
39064,GH0348456,GH,2018-12-03,2018-12-01,2018,GH0348456_2018_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/GH0348456_S2_201...
39071,GH0348456,GH,2022-11-17,2022-12-01,2022,GH0348456_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/GH0348456_S2_202...
39065,TZ0369306,TZ,2017-07-31,2017-08-01,2017,TZ0369306_2017_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/TZ0369306_S2_201...
39072,TZ0369306,TZ,2018-11-28,2018-12-01,2018,TZ0369306_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/TZ0369306_S2_201...
39066,ZM0194444,ZM,2018-07-31,2018-08-01,2018,ZM0194444_2018_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/ZM0194444_S2_201...
39073,ZM0194444,ZM,2021-08-02,2021-08-01,2021,ZM0194444_S2L2A_nomask.tif,mappingafrica-s2-256/window_b/ZM0194444_S2_202...


Merge back fixed S2 catalog with Mapping Africa catalog

In [ ]:
ma_catalog = pd.read_csv("../data/mappingafrica-catalog.csv")
ma_catalog["year"] = ma_catalog["window_b"].apply(
    lambda x: 
    re.sub(".tif", "", os.path.basename(x).split("_")[1]).split("-")[0]
)

# set Nambia country value to NAM, to avoid NA issues
ma_catalog.loc[ma_catalog.country.isna(), "country"] = "NAM"
ma_catalog.query("country == 'NAM'")

ma_s2_winb_catalog = pd.merge(
    ma_catalog.drop(columns="window_b"), 
    s2_winb_catalog, 
    how="left", 
    on=["name", "country", "year"]
)

cnames = ma_catalog.columns.to_list()
ma_s2_winb_catalog = ma_s2_winb_catalog[cnames + ["image"]]

ma_s2_winb_catalog.query("name.isin(@dup_names)").sort_values("name")
# ma_s2_winb_catalog.window_b.isna().any()
# ma_s2_winb_catalog.query("country=='NAM'")
# ma_s2_winb_catalog


,name,dataset,version,country,x,y,fld_prop,nonfld_prop,null_prop,window_a,window_b,mask,split,year,image
1445,GH0185923,mappingafrica,v1.3.0,GH,0.0265,9.4825,0.203165,0.796835,0.0,NaN,mappingafrica-s2-256/window_b/GH0185923_S2_201...,mappingafrica-256/labels/GH0185923_901445_2018...,train,2018,GH0185923_S2L2A_nomask.tif
13576,GH0185923,mappingafrica,v2.0.0,GH,0.0265,9.4825,0.467993,0.532007,0.0,NaN,mappingafrica-s2-256/window_b/GH0185923_S2_202...,mappingafrica-256/labels/GH0185923_17963_2021-...,train,2021,GH0185923_2021_S2L2A_nomask.tif
1765,GH0238214,mappingafrica,v1.3.0,GH,-1.8435,9.0725,0.702627,0.297373,0.0,NaN,mappingafrica-s2-256/window_b/GH0238214_S2_201...,mappingafrica-256/labels/GH0238214_901765_2018...,train,2018,GH0238214_S2L2A_nomask.tif
13664,GH0238214,mappingafrica,v2.0.0,GH,-1.8435,9.0725,0.346999,0.653001,0.0,NaN,mappingafrica-s2-256/window_b/GH0238214_S2_202...,mappingafrica-256/labels/GH0238214_20754_2021-...,train,2021,GH0238214_2021_S2L2A_nomask.tif
2119,GH0348456,mappingafrica,v1.3.0,GH,-0.1185,8.1925,0.181561,0.818439,0.0,NaN,mappingafrica-s2-256/window_b/GH0348456_S2_201...,mappingafrica-256/labels/GH0348456_902119_2018...,train,2018,GH0348456_2018_S2L2A_nomask.tif
13824,GH0348456,mappingafrica,v2.0.0,GH,-0.1185,8.1925,0.291255,0.708745,0.0,NaN,mappingafrica-s2-256/window_b/GH0348456_S2_202...,mappingafrica-256/labels/GH0348456_11777_2022-...,train,2022,GH0348456_S2L2A_nomask.tif
3892,TZ0369306,mappingafrica,v1.3.0,TZ,33.6265,-2.8375,0.821628,0.178372,0.0,NaN,mappingafrica-s2-256/window_b/TZ0369306_S2_201...,mappingafrica-256/labels/TZ0369306_903892_2018...,train,2018,TZ0369306_S2L2A_nomask.tif
31351,TZ0369306,mappingafrica,v2.0.0,TZ,33.6265,-2.8375,0.723892,0.276108,0.0,NaN,mappingafrica-s2-256/window_b/TZ0369306_S2_201...,mappingafrica-256/labels/TZ0369306_25466_2017-...,train,2017,TZ0369306_2017_S2L2A_nomask.tif
4452,ZM0194444,mappingafrica,v1.3.0,ZM,31.4965,-9.7625,0.261659,0.738341,0.0,NaN,mappingafrica-s2-256/window_b/ZM0194444_S2_202...,mappingafrica-256/labels/ZM0194444_904452_2021...,train,2021,ZM0194444_S2L2A_nomask.tif
36446,ZM0194444,mappingafrica,v2.0.0,ZM,31.4965,-9.7625,0.127989,0.872011,0.0,NaN,mappingafrica-s2-256/window_b/ZM0194444_S2_201...,mappingafrica-256/labels/ZM0194444_26365_2018-...,train,2018,ZM0194444_2018_S2L2A_nomask.tif


Rename and copy files.

In [4]:
img_dir = "/Users/LEstes/data/labels/cropland/"
    
os.makedirs(Path(img_dir) / "mappingafrica-s2-256" / "window_b", exist_ok=True)

# Ensure logs directory exists and open the log file for appending
log_path = Path("../logs/s2image_rename.txt")
log_path.parent.mkdir(parents=True, exist_ok=True)

with open(log_path, "a") as log:
    for i, row in ma_s2_winb_catalog.iterrows():
        fnin = f"mappingafrica-s2-256/images/{row['image']}"
        fnout = row["window_b"]
        # print(fnin)

        src = Path(img_dir) / fnin
        dst = Path(img_dir) / fnout

        # Ensure input file exists before copying
        if src.exists() and not dst.exists():
            shutil.copy(src, dst)
            msg = f"Copied: {src} → {dst}\n"
            log.write(msg)
            # print(msg.strip())
        elif not src.exists():
            msg = f"⚠️ Source missing: {src}\n"
            log.write(msg)
            # print(msg.strip())
        else:
            # file already exists at destination
            msg = f"⚠️ File already exists: {dst}\n"
            log.write(msg)
            # print(msg.strip())


Save out Mapping Africa only version 

In [7]:
ma_s2_winb_catalog = ma_s2_winb_catalog[
    ["name", "dataset", "version", "country", "x", "y", "fld_prop", 
     "nonfld_prop", "null_prop", "window_a", "window_b", "mask", "split"]
]
ma_s2_winb_catalog

ma_s2_winb_catalog.to_csv(
    "../data/mappingafrica-catalog-s2.csv"
)

FTW long 2 and MA S2

In [10]:
ftw_long2 = pd.read_csv("../data/ftw-catalog-long2.csv")
catalog_long = pd.concat([ftw_long2, ma_s2_winb_catalog], axis=0, 
                         ignore_index=True)
catalog_long.to_csv(
    "../data/ftw-ma-s2-combined-catalog-long2.csv", 
    index=False
)

## Two Time-Points

### Integrating Window A for MA chips

In [60]:
s2_dir = "/Users/LEstes/data/labels/cropland/mappingafrica-s2-256/images"
fns = [f for f in os.listdir(s2_dir)]

s2_wina_catalog = [{"name": f.split("_")[0], "pl_date": f.split("_")[1], 
                    "year": f.split("_")[1].split('-')[0], 
                    "image": f} for f in fns if f.endswith(".tif")]
s2_wina_catalog = pd.DataFrame(s2_wina_catalog)

s2_wina_catalog["window_a"] = s2_wina_catalog.apply(
    lambda x: 
    (f"mappingafrica-s2-256/window_a/"\
     f"{x['name']}_S2_{'-'.join(x.image.split('_')[2].split('-')[:2])}.tif"), 
    axis=1
)
s2_wina_catalog["country"] = s2_wina_catalog["name"].str[:2]
mask = s2_wina_catalog["country"] == "NA"
s2_wina_catalog.loc[mask, "country"] = "NAM"
s2_wina_catalog[s2_wina_catalog["country"] == "NAM"]
# s2_wina_catalog

,name,pl_date,year,image,window_a,country
86,NA0399530,2021-06,2021,NA0399530_2021-06_2021-03-02_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0399530_S2_202...,NAM
139,NA0038424,2023-07,2023,NA0038424_2023-07_2023-02-25_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0038424_S2_202...,NAM
197,NA1338615,2022-07,2022,NA1338615_2022-07_2022-02-25_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA1338615_S2_202...,NAM
529,NA0060609,2022-11,2022,NA0060609_2022-11_2022-08-17_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0060609_S2_202...,NAM
1180,NA0609528,2023-06,2023,NA0609528_2023-06_2023-02-20_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0609528_S2_202...,NAM
...,...,...,...,...,...,...
38612,NA0262423,2017-08,2017,NA0262423_2017-08_2017-01-30_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0262423_S2_201...,NAM
38740,NA0305437,2020-07,2020,NA0305437_2020-07_2020-02-13_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0305437_S2_202...,NAM
38742,NA0876729,2019-08,2019,NA0876729_2019-08_2019-02-16_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0876729_S2_201...,NAM
38951,NA0294018,2017-08,2017,NA0294018_2017-08_2017-03-05_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/NA0294018_S2_201...,NAM


In [ ]:
dup_names = s2_wina_catalog.name[s2_wina_catalog.name.duplicated()].to_list()
cnames = ma_catalog.columns.to_list()
ma_catalog = pd.read_csv("../data/mappingafrica-catalog.csv")
ma_catalog["year"] = ma_catalog["window_b"].apply(
    lambda x: 
    re.sub(".tif", "", os.path.basename(x).split("_")[1]).split("-")[0]
)

# # set Nambia country value to NAM, to avoid NA issues
ma_catalog.loc[ma_catalog.country.isna(), "country"] = "NAM"
ma_catalog.query("country == 'NAM'")

ma_s2wina_plwinb_catalog = pd.merge(
    ma_catalog.drop(columns="window_a"), 
    s2_wina_catalog, 
    how="left", 
    on=["name", "country", "year"]
)
ma_s2wina_plwinb_catalog
# ma_s2wina_plwinb_catalog.query("name.isin(@dup_names)").sort_values("name")

,name,dataset,version,country,x,y,fld_prop,nonfld_prop,null_prop,window_b,mask,split,year,pl_date,image,window_a
0,BF0742031,mappingafrica,v1.3.0,BF,-0.3185,11.1875,0.125957,0.874043,0.0,mappingafrica-256/images/BF0742031_2018-12.tif,mappingafrica-256/labels/BF0742031_900000_2018...,validate,2018,2018-12,BF0742031_2018-12_2018-06-26_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/BF0742031_S2_201...
1,BF0743326,mappingafrica,v1.3.0,BF,-0.3285,11.1825,0.029456,0.970544,0.0,mappingafrica-256/images/BF0743326_2018-12.tif,mappingafrica-256/labels/BF0743326_900001_2018...,train,2018,2018-12,BF0743326_2018-12_2018-06-26_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/BF0743326_S2_201...
2,BF0743327,mappingafrica,v1.3.0,BF,-0.3235,11.1825,0.210957,0.789043,0.0,mappingafrica-256/images/BF0743327_2018-12.tif,mappingafrica-256/labels/BF0743327_900002_2018...,train,2018,2018-12,BF0743327_2018-12_2018-06-26_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/BF0743327_S2_201...
3,BF0743329,mappingafrica,v1.3.0,BF,-0.3135,11.1825,0.310746,0.689254,0.0,mappingafrica-256/images/BF0743329_2018-12.tif,mappingafrica-256/labels/BF0743329_900003_2018...,train,2018,2018-12,BF0743329_2018-12_2018-06-26_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/BF0743329_S2_201...
4,BF0776469,mappingafrica,v1.3.0,BF,-0.4485,11.0475,0.089226,0.910774,0.0,mappingafrica-256/images/BF0776469_2018-12.tif,mappingafrica-256/labels/BF0776469_900004_2018...,train,2018,2018-12,BF0776469_2018-12_2018-06-26_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/BF0776469_S2_201...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39071,ZW1290563,mappingafrica,v2.0.0,ZW,29.6615,-21.7575,0.424884,0.575116,0.0,mappingafrica-256/images/ZW1290563_2022-06.tif,mappingafrica-256/labels/ZW1290563_16779_2022-...,train,2022,2022-06,ZW1290563_2022-06_2022-02-20_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/ZW1290563_S2_202...
39072,ZW1323816,mappingafrica,v2.0.0,ZW,29.6465,-22.0525,0.000000,1.000000,0.0,mappingafrica-256/images/ZW1323816_2022-06.tif,mappingafrica-256/labels/ZW1323816_8232_2022-0...,train,2022,2022-06,ZW1323816_2022-06_2022-02-25_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/ZW1323816_S2_202...
39073,ZW1327144,mappingafrica,v2.0.0,ZW,31.0865,-22.0825,0.631059,0.368941,0.0,mappingafrica-256/images/ZW1327144_2022-06.tif,mappingafrica-256/labels/ZW1327144_100281_2022...,train,2022,2022-06,ZW1327144_2022-06_2022-02-20_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/ZW1327144_S2_202...
39074,ZW1330078,mappingafrica,v2.0.0,ZW,31.4265,-22.1125,0.733777,0.266223,0.0,mappingafrica-256/images/ZW1330078_2018-08.tif,mappingafrica-256/labels/ZW1330078_100282_2018...,train,2018,2018-08,ZW1330078_2018-08_2018-03-08_S2L2A_nomask.tif,mappingafrica-s2-256/window_a/ZW1330078_S2_201...


Rename chips

In [63]:
img_dir = "/Users/LEstes/data/labels/cropland/"
    
# os.makedirs(Path(img_dir) / "mappingafrica-s2-256" / "window_b", exist_ok=True)

# Ensure logs directory exists and open the log file for appending
log_path = Path("../logs/s2image_wina_rename.txt")
log_path.parent.mkdir(parents=True, exist_ok=True)

with open(log_path, "a") as log:
    for i, row in ma_s2wina_plwinb_catalog.iterrows():
        fnin = f"mappingafrica-s2-256/images/{row['image']}"
        fnout = row["window_b"]
        # print(fnin)

        src = Path(img_dir) / fnin
        dst = Path(img_dir) / fnout

        # Ensure input file exists before copying
        if src.exists() and not dst.exists():
            shutil.copy(src, dst)
            msg = f"Copied: {src} → {dst}\n"
            log.write(msg)
            # print(msg.strip())
        elif not src.exists():
            msg = f"⚠️ Source missing: {src}\n"
            log.write(msg)
            # print(msg.strip())
        else:
            # file already exists at destination
            msg = f"⚠️ File already exists: {dst}\n"
            log.write(msg)
            # print(msg.strip())


Save out catalog, MA with S2 Window A, Planet Window B

In [ ]:
(ma_s2wina_plwinb_catalog[[c for c in cnames if c != "year"]]
 .sort_values("name")
 .to_csv("../data/mappingafrica-catalog-s2wina-plwinb.csv", index=False))

### Remaining catalogs 

- Mapping Africa with S2 Window A, S2 Window B
- Full Catalog with S2 Window A, Planet Window B
- Full Catalog with S2 Window A, S2 Window B

In [182]:
ftw = pd.read_csv("../data/ftw-catalog2.csv")
# ma_s2winb = pd.read_csv("../data/mappingafrica-catalog-s2.csv")
# ma_s2winb = ma_s2winb.drop(columns=ma_s2winb.columns[0])
# ma_s2winb.to_csv("../data/mappingafrica-catalog-s2.csv", index=False)

ma_s2winb = pd.read_csv("../data/mappingafrica-catalog-s2.csv")
# ma_s2winb["year"] = ma_s2winb["window_b"].apply(
#     lambda x: 
#     re.sub(".tif", "", os.path.basename(x).split("_")[2]).split("-")[0]
# )
ma_s2wina_plwinb = pd.read_csv(
    "../data/mappingafrica-catalog-s2wina-plwinb.csv"
)
ma_s2wina_plwinb["year"] = ma_s2wina_plwinb["window_b"].apply(
    lambda x: 
    re.sub(".tif", "", os.path.basename(x).split("_")[1]).split("-")[0]
)
ma_s2wina_s2winb = pd.merge(
    ma_s2winb.drop(columns={"window_a"}),
    ma_s2wina_plwinb[["name", "country", "version", "year", "window_a"]], 
    how="left",
    on=["name", "country", "version"]
)[cnames]

# ma_s2winb.query("name == 'AO1648328'"), 
# ma_s2wina_s2winb.query("name == 'AO1648328'")

# # MA S2 winA winB
# ma_s2wina_s2winb = pd.merge(
#     ma_s2winb.drop(columns={"window_a"}),
#     ma_s2wina_plwinb_catalog[["name", "year", "country", "window_a"]], 
#     how="left",
#     on=["name", "country", "year"]
# )[cnames]
# ma_s2wina_s2winb.query("name.isin(@dup_names)").sort_values("name")

(ma_s2wina_s2winb
 .drop(columns={"year"})
 .sort_values("name")
 .to_csv("../data/mappingafrica-catalog-s2wina-s2winb.csv", index=False))

# Full S2 with MA S2 winA, Pl winB
full_s2wina_plwinb = pd.concat(
    [ftw,
     ma_s2wina_plwinb_catalog[[c for c in cnames if c != "year"]]\
        .sort_values("name")]
)
full_s2wina_plwinb.to_csv(
    "../data/ftw-ma-s2wina-plwinb-combined.csv",
    index=False
)

# Full S2 with MA S2 winA, Pl winB
full_s2wina_s2winb = pd.concat(
    [ftw,
     (ma_s2wina_s2winb
      .drop(columns={"year"})
      .sort_values("name"))]
)
full_s2wina_s2winb.to_csv(
    "../data/ftw-ma-s2wina-s2winb-combined.csv",
    index=False
)

# Experiments

## Functions

In [101]:
def write_yaml(template_path: str, output_path: str, updates: dict = None):
    """
    Write a YAML file from a template file, with optional updates.

    Args:
        template_path (str): Path to the base YAML template file.
        output_path (str): Path to the output YAML file.
        updates (dict, optional): Dictionary of keys/values to update.
    """

    def recursive_update(d, u):
        for k, v in u.items():
            if isinstance(v, dict) and isinstance(d.get(k), dict):
                recursive_update(d[k], v)
            else:
                d[k] = v

    with open(template_path, 'r') as f:
        config = yaml.safe_load(f)
        if updates:
            recursive_update(config, updates)

    class IndentDumper(yaml.SafeDumper):
        def increase_indent(self, flow=False, indentless=False):
            return super().increase_indent(flow, False)

    # custom representer for lists
    def represent_list(dumper, data):
        # flow style only if all elements are scalars
        if all(isinstance(x, (str, int, float, bool, type(None))) for x in data):
            return dumper.represent_sequence("tag:yaml.org,2002:seq", data, 
                                             flow_style=True)
        else:
            return dumper.represent_sequence("tag:yaml.org,2002:seq", data, 
                                             flow_style=False)

    IndentDumper.add_representer(list, represent_list)

    with open(output_path, 'w') as f:
        yaml.dump(
            config,
            f,
            Dumper=IndentDumper,
            default_flow_style=False,  # keep dicts block-style
            sort_keys=False,
            indent=2,
            allow_unicode=True
        )


## Naming conventions

Model config files are general named according to the following recipe:

- `<model>_<backbone>-<loss>-<norm>-<augmentation>-<epochs><-patience>-<catalog>`

Although this recipe has varied a bit as experiments have evolved, primarily in the early phase where different settings were being evaluated, and as efforts to shorten names were applied (the config is applied to model outputs folders and metrics files, which become cumbersome).

The pattern above is generally for models trained on just the FTW dataset, where there were different permutations of the single time point catalog initially (window_b only and window_b + window_a combined under window_b [long catalogs]).  

For other catalogs--full and Mapping Africa only--the recipe is as follows:

- `<full|ma>-<model>_<backbone>-<loss>-<norm>-<augmentation>-<epochs><-patience>-<catalog>`

In all cases, there are variations within most components also. 

- For FTW baseline architecture models (U-Net with EfficientNet-b3), `<model>_<backbone>` is either simply `ftwbaseline` or `ftwbase`
- For the model approximating our standard Mapping Africa architecture (U-Net with VGG19), `<model>_<backbone>` is either `ma-approximate` or `ma-approx` or `ma-approxbase`
- For other variants, the full model and backbone names are used, e.g., `unet_cvnxt_sm` or `unet_cvnxt_sm` for U-Net with ConvNeXt Small backbone pretrained on ImageNet-22K. 
- Losses vary, but are fairly self-explanatory, although locally-weighted tversky focal is often not mentioned (in ma-approx models), where it is standard, or called `localtversky` or `loctver`, and finally `ltfl`.  Similarly weighted cross-entropy is left unmentioned for FTW baselines models. The loss in these cases is only made explicit when different. 
- The general augmentation are abbreviated out separately, specifying additional augmentations added on top of what FTW used by default (rotation, v and h flips, sharpness). These include `rescale` or `rscl` and `rscl2`, which is rescaling with different parameters, and `brightness` or `brgt`. 
    - We have settled now onto a standard package that adds brightness and rescaling with FTW settings, so names containg `sta` now indicate the final standard augmentation package. 
- Epochs are not listed unless longer than 100. Similarly with patience. As both of these should be matched, the standard now for trains longer than 100 is e.g. `ep300` or epochs of 300 with matching patience.
- `<catalog>` is not always provided either. Older models runs did not mention this, as they only used window_b. Newer FTW or full catalog runs end typically in `longcat` to show that they use a single window, but the model is shown both window A and B. Later experiments drop this again because all single time point runs involving the FTW catalog now use longcat. 






## Single time point

### FTW catalog

#### FTW baseline 
All experiments here are FTW baseline model, window B only

##### Setup

Below we set up a yaml for each experiment. Provide the following:

- `cfg_name`: name of the config/experiment file (without .yaml)
- `update`: dictionary of changes to make to the base config

Also define a global `home_dir` for the path to the repo containing the catalog. That's done once in the first cell. 

In [8]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog2.csv",
        )
    )
)

##### # 1 Default

With updated optimal weights added

In [ ]:
cfg_name = "ftwbaseline-exp1"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 2 Locally-weighted tversky focal loss


In [93]:
cfg_name = "ftwbaseline-localtversky-exp2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 3 Local min-max normalization, across bands


In [94]:
cfg_name = "ftwbaseline-minmax_lab-exp3"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    global_stats=None,
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 3a Local min-max normalization, across bands, 1% clip

In [19]:
cfg_name = "ftwbaseline-minmax_lab-exp3a"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    img_clip_val=1,
    global_stats=None,
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 4 Global min-max normalization, across bands

Using the 1st and 99th percentiles as min-max. 


In [18]:
cfg_name = "ftwbaseline-minmax_gab-exp4"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

# Ensure global_stats exists before popping
global_stats = update["data"]["init_args"].get("global_stats", {})
global_stats.pop("mean", None)
global_stats.pop("std", None)
update["data"]["init_args"]["global_stats"] = global_stats

update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### # 4a Per band Z-value, global stats

In [20]:
cfg_name = "ftwbaseline-zvalue_gpb-exp4a"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    normalization_stat_procedure="gpb",
    global_stats={"mean": [874.538, 876.9152, 641.7087, 2925.554], 
                  "std": [759.7574, 648.3951, 619.3338, 1083.3489]},
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 5 - photometric augmentations

In [95]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-photometric-exp5"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["brightness", "contrast", "gaussian_noise"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 6 - satslidemix

In [96]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-satslide-exp6"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["satslidemix"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 7 - rescale

In [97]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-rescale-exp7"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["rescale"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 8 - Tversky and min-max GAB

In [13]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-exp8"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA Approximate baseline model

In [10]:
cfg_name = "ftw-ma-approximate-exp1"
# update = dict(
#     data=dict(
#         init_args=dict(
#             catalog=f"{home_dir}/"\
#                 "ftw-mappingafrica-integration/data/ftw-catalog2.csv"
#         )
#     )
# )
# update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
# write_yaml("../configs/ma-approximate-baseline.yaml", 
#            f"../configs/{cfg_name}.yaml", 
#            updates=update)

augs = ["rotation", "hflip", "vflip", "sharpness"]
# cfg_name = "ma-approximate-baseline"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "contrast", "gaussian_noise", "rescale", "gamma"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

### Mapping Africa catalog


In [43]:
home_dir = "~/projects"
augs = ["rotation", "hflip", "vflip", "sharpness"]
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/mappingafrica-catalog.csv"
        )
    )
)

#### MA Approximate baseline 

We will train a model close to the variant we used on Planet imagery for mapping countries in Africa, initially trained on just the Mapping Africa labels.

##### Near default

These settings are about as close as possible to those originally used

In [12]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approximate-baseline"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "contrast", "gaussian_noise", "rescale", "gamma"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approximate with only brightness and rescale

Brightness now has 50% probability, and rescale parameters are now different tracking FTW's. This steps it down a bit now. 

In [15]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approxbase-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approximate, minmax GAB, brightness and rescale

Using MA global stats, between 1st and 99th percentiles: 

- min: 203.544
- max: 4590.354

In [8]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approxbase-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approximate, SMP tversky, minmax GAB, brightness and rescale


In [15]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approx-smptver-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approx, logcoshdice, minmax GAB, brightness and rescale

In [ ]:
cfg_name = "ma-approx-logcoshdice-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### FTW baseline 

##### # 1 Standard

In [ ]:
cfg_name = "ma-ftwbaseline-exp1"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    catalog=f"{home_dir}/"\
        f"ftw-mappingafrica-integration/data/mappingafrica-catalog.csv"   
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 2 Min-max, lab

In [ ]:
cfg_name = "ma-ftwbaseline-exp2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    global_stats=None,
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline with local tversky

Let's put this more on an equal footing with the MA approximate baseline with local tversky loss and similar augmentations. 



In [19]:
cfg_name = "ma-ftwbase-localtversky-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, local tversky, minmax GAB, brightness and rescale

In [5]:
cfg_name = "ma-ftwbase-loctversky-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, logcoshdice, minmax GAB, brightness and rescale

In [7]:
cfg_name = "ma-ftwbase-logcoshdice-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, SMP tversky, minmax GAB, brightness and rescale

In [ ]:
cfg_name = "ma-ftwbase-smptversky-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, tversky focal, standard augmentations

In [9]:
cfg_name = "ma-ftwbase-tfl-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tverskyfocal", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, tversky focal CE, standard augmentations

In [47]:
cfg_name = "ma-ftwbase-tflce-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        loss="tverskyfocalce", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, local tversky focal CE, standard augmentations

In [12]:
cfg_name = "ma-ftwbase-localtflce-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tverskyfocalce", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, local tversky focal, standard augmentations

This one is a redo to check that new version gives the same results as before.

In [13]:
cfg_name = "ma-ftwbase-localtfl-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### ConvNeXt, local tversky, minmax GAB, brightness and rescale

In [30]:
cfg_name = "ma-unet_cvnxt_sm-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=100,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


#### Efficientnet-b7, local tversky, minmax GAB, brightness and rescale

In [32]:
ep = 100
cfg_name = "ma-unet-effb7-ltfl-mnmx_gab-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="efficientnet-b7",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

### Full catalog

Setting up runs that will pull from the full catalog, starting with the FTW baseline model.

We'll also specify some validation sets to evaluate results. To start we will use the separated FTW and Mapping Africa "global" validation samples, without confining them to specific countries. 

In [ ]:
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-mappingafrica-combined-catalog2.csv"   
        )
    )
)


#### FTW Baseline


##### # 1 Standard settings

In [ ]:
cfg_name = "fullcat-ftwbaseline-exp1"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 2 LAB

In [120]:
cfg_name = "fullcat-ftwbaseline-exp2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    global_stats=None,
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 3 Locally-weighted tversky focal loss



In [14]:
cfg_name = "fullcat-ftwbaseline-exp3"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA approximate baseline

In [ ]:
cfg_name = "fullcat-ma-approximate-baseline-exp1"
update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-mappingafrica-combined-catalog2.csv"   
        )
    )
)
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
write_yaml("../configs/ma-approximate-baseline.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

## Single time point, long catalog

Re-running training for the full catalog and FTW only versions using the new long catalog.



### FTW catalog

In [33]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog-long2.csv",
        )
    )
)

#### FTW baseline

##### Locally-weighted tversky focal loss, redo

In [16]:
cfg_name = "ftwbaseline-localtversky-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky and min-max GAB, redo

In [21]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky, min-max GAB, photometric augmentations

In [26]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-localtversky-minmax_gab-photometric-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["brightness", "contrast", "gaussian_noise", "gamma"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky, min-max GAB, photometric augmentations, rescale 

In [28]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-photometric-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness", "contrast", "gaussian_noise", 
                     "gamma"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky, min-max GAB, brightness only, rescale

Back to ftw-baseline. Note I tweaked the augmentation code to make this align now with FTW's brightness augmentation approach. 

In [32]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-brightness-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### SMP Tversky, min-max GAB, brightness only, rescale

In [6]:
cfg_name = "ftwbase-smptversky-mnmx_gab-bright-rescale-longcat"
augs = ["rotation", "hflip", "vflip", "sharpness"]
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="tversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Dice, min-max GAB, brightness only, rescale

In [9]:
cfg_name = "ftwbase-dice-mnmx_gab-bright-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="dice"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### logCoshDice, min-max GAB, brightness only, rescale

In [10]:
cfg_name = "ftwbase-logcoshdice-mnmx_gab-bright-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="logcoshdice"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### ConvNeXt backbone

In [20]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog-long2.csv",
        )
    )
)

##### Full augs, local-tversky

We'll try out ConvNeXt small, pre-trained on ImageNet-22k. **Note**: the yaml should have been named differently (convnext_small-...). Also note this one had the wrong normalization stats procedure set (should have been gab, was lab)

In [ ]:
cfg_name = "ftwbaseline-tversky-allaugs-convnext_small_in22k-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["rescale", "brightness", "contrast", "gaussian_noise", "gamma"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, local tversky, brightness & rescale, 300 epochs

Ditching the additional photometrics augmentations since those are probably disappearing, and so the main thing here is to see if the model keeps learning. Validation curves from the model above suggested it had substantial improvements left. GAB normalization added as before. 

Note: I manually updated this one to 400 epochs after seeing continued gains at 300 epochs, and then again to 500 epochs.

In [23]:
cfg_name = "unet_cvnxt_sm_in22k-loctversky-brgt-rescl-e300-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=300,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, local tversky, brightness & rescale, 300 epochs, different patience

The ups and downs in learning on the long run from the one above seem to be resetting learning rate due to patience of 100 and cosine annealing. Will match patience to epochs to make continual progress. 

In [16]:
cfg_name = "unet_cvnxt_sm-loctver-brgt-rescl-e300-p300-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=300,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
        patience=300,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, local tversky, brightness & rescale, 400 epochs

The ups and downs in learning on the long run from the one above seem to be resetting learning rate due to patience of 100 and cosine annealing. Will match patience to epochs to make continual progress. 

In [22]:
cfg_name = "unet_cvnxt_sm-ltfl-sta-ep400-longcat"
ep = 400
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, logcoshdice, brightness & rescale, 400 epochs

Trying best performing logcoshdice, which seems better than local tversky, although weak on object recall. 

In [13]:
cfg_name = "unet_cvnxt_sm_in22k-logcoshdice-brgt-rescl-e400-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=400,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, logcoshdice, brightness & rescale, 300 epochs, 300 patience

Above with LR not resetting due to patience being too short.

In [17]:
cfg_name = "unet_cvnxt_sm-logcoshdice-brgt-rescl-e400-p300-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=400,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
        backbone="tu-convnext_small.fb_in22k",
        patience=300,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


#### EfficientNet-b7, 400 epochs

In [34]:
ep = 400
cfg_name = "unet-effb7-ltfl-mmgab-sta-ep400-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="efficientnet-b7",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

### Mapping Africa, S2

In [16]:
home_dir = "~/projects"
augs = ["rotation", "hflip", "vflip", "sharpness"]
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                f"ftw-mappingafrica-integration/data/"\
                "mappingafrica-catalog-s2.csv"   
        )
    )
)


##### FTW baseline, local tversky, mmGAB, sta

In [19]:
cfg_name = "mas2-ftwbase-localtfl-mngab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [213.5], "max": [6079.5], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

### Full catalog

In [6]:
home_dir = "~/projects"
augs = ["rotation", "hflip", "vflip", "sharpness"]
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                f"ftw-mappingafrica-integration/data/"\
                    "ftw-mappingafrica-combined-catalog-long2.csv"   
        )
    )
)


#### FTW base, local tversky, GAB, brightness & rescale, 100 epochs

Maybe extend longer depending on curves. Keeping patience at 100. 

In [23]:
cfg_name = "full-ftwbase-loctver-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### FTW base, SMP tversky, GAB, brightness & rescale, 100 epochs



In [19]:
cfg_name = "full-ftwbase-smptver-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA approximate, local tversky, GAB, brightness & rescale, 100 epochs


In [25]:
cfg_name = "full-ma-approx-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA approximate, smp tversky, GAB, brightness & rescale, 100 epochs


In [17]:
cfg_name = "full-ma-approx-smptver-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### ConvNext, local tversky, brightness & rescale, 300 epochs, 300 patience


In [27]:
ep = 400
cfg_name = "full-unet_cvnxt_sm-ltfl-sta-ep400"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3],
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


#### EfficientNet-b7, 400 epochs

In [7]:
ep = 400
cfg_name = "full-unet-effb7-ltfl-mmgab-sta-ep400"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="efficientnet-b7",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3],
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

# Experiments, revised

The preceding were initial experiments, prior to most FTW model release. Reorganize these to tidy up config directories, and restart experiments below here.  

In [ ]:
# os.makedirs("../configs/initial")
for f in os.listdir("../configs/"): 
    if f.endswith("yaml"):
        shutil.move(
            Path("../configs/") / f, 
            Path("../configs/initial") / f
        ) 

### Image stats

In [95]:
band_stats = dict(
    t1 = dict(
        ma = {"min": [203.5], "max": [4590.4]}, 
        ma_s2 = {"min": [213.5], "max": [6079.5]},
        ftw = {"min": [68.5], "max": [5772.3]},  # this should be same as t2
        full = {"min": [89.1], "max": [5528.3]},
        full_s2 = {"min": [87.7], "max": [5714]}
    ),
    t2 = dict(
        ma_s2pl = {"min": [169], "max": [5988]},
        ma_s2s2 = {"min": [158], "max": [6534]},
        ftw = {"min": [83], "max": [5637]},
        full_s2pl = {"min": [91], "max": [5708]},
        full_s2s2 = {"min": [89], "max": [5879]}
    )
)

## Single time point

### Full train

#### FTW long

In [6]:
os.makedirs("../configs/1timepoint")

In [57]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog-long2.csv",
        )
    )
)

##### PRUE-like, localtversky (default model)

Efficient Net B7, standard augmenations, min-max global all band. 

In [40]:
cfg_name = "ftw-unet-effnb7-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

##### PRUE-like, logcoshdice


In [41]:
cfg_name = "ftw-unet-effnb7-logcoshdice-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="logcoshdice"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

##### ConvNext-base, local TFL

From [here](https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/convnext.py). To test something a bit larger than EfficientNet-b7

In [59]:
cfg_name = "ftw-unet-cvnxt-base-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_base.fb_in22k",
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

#### MA

In [42]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/mappingafrica-catalog.csv",
        )
    )
)

##### PRUE-like, localtversky (default)

Efficient Net B7, standard augmenations, min-max global all band. 

In [43]:
cfg_name = "ma-unet-effnb7-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

##### PRUE-like, logcoshdice


In [44]:
cfg_name = "ma-unet-effnb7-logcoshdice-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="logcoshdice"))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

#### MA-S2

In [50]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/mappingafrica-catalog-s2.csv",
        )
    )
)

##### PRUE-like, localtversky (default)

Efficient Net B7, standard augmenations, min-max global all band. 

In [51]:
cfg_name = "mas2-unet-effnb7-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma_s2']) 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

#### Full

In [ ]:
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-mappingafrica-combined-catalog2.csv"   
        )
    )
)

##### PRUE-like, localtversky (default)

In [53]:
cfg_name = "full-unet-effnb7-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['full_s2']) 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

#### Full S2

In [54]:
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=(
                f"{home_dir}/ftw-mappingafrica-integration/data/"\
                "ftw-ma-s2-combined-catalog-long2.csv"
            )   
        )
    )
)
base_update

{'data': {'init_args': {'catalog': '~/projects/ftw-mappingafrica-integration/data/ftw-ma-s2-combined-catalog-long2.csv'}}}

##### PRUE-like, localtversky (default)

In [55]:
cfg_name = "fulls2-unet-effnb7-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['full_s2']) 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

## Two time points 

### FTW only 

To have a PRUE-like model for comparison. 



In [112]:
home_dir = "~/projects"
cfg_name = "ftw2w-unet-effnb7-ltfl-sta-ep100"
update = dict()
update.setdefault("trainer", {})
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(in_channels=8))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t2']['ftw'],
        random_shuffle=True,
        temporal_options="stacked"
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/2timepoints/{cfg_name}.yaml", 
           updates=update)

### MA S2 Win A, Planet Win B

With PRUE-like model

In [113]:
home_dir = "~/projects"
cfg_name = "ma2w-s2pl-unet-effnb7-ltfl-sta-ep100"
update = dict()
update.setdefault("trainer", {})
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(in_channels=8))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t2']['ma_s2pl'],
        random_shuffle=True,
        temporal_options="stacked",
        catalog=(
            f"{home_dir}/ftw-mappingafrica-integration/data/"\
            "mappingafrica-catalog-s2wina-plwinb.csv"
        )   
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/2timepoints/{cfg_name}.yaml", 
           updates=update)

### MA S2 Win A, S2 Win B

With PRUE-like model

In [ ]:
cfg_name = "ma2w-s2s2-unet-effnb7-ltfl-sta-ep100"
update = dict()
update.setdefault("trainer", {})
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(in_channels=8))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t2']['ma_s2s2'],
        random_shuffle=True,
        temporal_options="stacked",
        catalog=(
            f"{home_dir}/ftw-mappingafrica-integration/data/"\
            "mappingafrica-catalog-s2wina-s2winb.csv"
        )   
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/2timepoints/{cfg_name}.yaml", 
           updates=update)

### Full with MA S2 Win A, Planet Win B

In [159]:
cfg_name = "full2w-s2pl-unet-effnb7-ltfl-sta-ep100"
update = dict()
update.setdefault("trainer", {})
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(in_channels=8))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t2']['full_s2pl'],
        random_shuffle=True,
        temporal_options="stacked",
        catalog=(
            f"{home_dir}/ftw-mappingafrica-integration/data/"\
            "ftw-ma-s2wina-plwinb-combined.csv"
        )   
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/2timepoints/{cfg_name}.yaml", 
           updates=update)

### Full with MA S2 Win A, S2 Win B

In [160]:
cfg_name = "full2w-s2s2-unet-effnb7-ltfl-sta-ep100"
update = dict()
update.setdefault("trainer", {})
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(in_channels=8))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t2']['full_s2s2'],
        random_shuffle=True,
        temporal_options="stacked",
        catalog=(
            f"{home_dir}/ftw-mappingafrica-integration/data/"\
            "ftw-ma-s2wina-s2winb-combined.csv"
        )   
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/2timepoints/{cfg_name}.yaml", 
           updates=update)

## Transfer

Fine-tuning models trained on one dataset onto another

### Single time point

#### FTW -> MA

In [191]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/mappingafrica-catalog.csv",
        )
    ), 
    # model=dict(init_args=dict())
)

##### PRUE-like, localtversky (default)

- Both decoder and encoder unfrozen
- 50 epochs
- Low learning rate: 1e-04
- Last checkpoint from `ftw-unet-effnb7-ltfl-sta-ep50`

In [125]:
cfg_name = "ftw-to-ma-unet-effnb7-ltfl-lr1e4-sta-ep50"
ep = 150
update = base_update.copy()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(lr="1e-4", patience=ep-100))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) # set stats to MA 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

##### Same variant, but LR 1e-5

In [126]:
cfg_name = "ftw-to-ma-unet-effnb7-ltfl-lr1e5-sta-ep50"
ep = 150
update = base_update.copy()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(lr="1e-5", patience=ep-100))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) # set stats to MA 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

##### Two-stage PRUE-like, localtversky (default)

AI-recommended:

- Stage 1: Freeze decoder for 3 epochs, 1e-4 LR
- Stage 2: Unfreeze all 50 epochs, 1e-5 LR  

In [122]:
cfg_name = "ftw-to-ma-unet-effnb7-ltfl-sta-stg1"
ep = 103
update = base_update.copy()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(freeze_backbone=True, lr="1e-4"))
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) # set stats to MA 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

In [121]:
cfg_name = "ftw-to-ma-unet-effnb7-ltfl-sta-stg2"
ep = 153
update = base_update.copy()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)

# Ensure we update model.init_args keys without overwriting the whole section
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(lr="1e-5", patience=ep-103))

update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) # set stats to MA 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

##### Two-stage 2

AI-recommended:

- Stage 1: Freeze decoder for 10 epochs, 1e-4 LR
- Stage 2: Unfreeze all 50 epochs, 1e-5 LR  

In [ ]:
cfg_name = "ftw-to-ma-unet-effnb7-ltfl-lr1e3-sta-stg1-ep10"
ep = 110
update = dict()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(
    dict(freeze_backbone=True, lr="1e-3", patience=ep-100)
)
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) # set stats to MA 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

In [198]:
cfg_name = "ftw-to-ma-unet-effnb7-ltfl-lr1e4-sta-stg2-ep50"
ep = 160
update = dict()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)

# Ensure we update model.init_args keys without overwriting the whole section
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(lr="1e-4", patience=ep-110))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(global_stats=band_stats['t1']['ma']) # set stats to MA 
)              
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

#### MA -> FTW

In [186]:
cfg_name = "ma-to-ftw-unet-effnb7-ltfl-lr1e5-sta-ep50"
ep = 150
update = dict()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(lr="1e-5", patience=ep-100))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t1']['ftw'],
        catalog=(
            f"{home_dir}/ftw-mappingafrica-integration/data/"\
            "ftw-catalog-long2.csv"
        )   
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

#### Full -> MA

In [187]:
cfg_name = "full-to-ma-unet-effnb7-ltfl-lr1e5-sta-ep50"
ep = 150
update = dict()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(lr="1e-5", patience=ep-100))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t1']['ma'],
        catalog=(
            f"{home_dir}/ftw-mappingafrica-integration/data/"\
            "mappingafrica-catalog.csv"
        )   
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)

#### Full -> FTW

In [188]:
cfg_name = "full-to-ftw-unet-effnb7-ltfl-lr1e5-sta-ep50"
ep = 150
update = dict()
update["trainer"] = dict(
    default_root_dir=f"~/working/models/{cfg_name}",
    max_epochs=ep
)
update.setdefault("model", {})
update["model"].setdefault("init_args", {})
update["model"]["init_args"].update(dict(lr="1e-5", patience=ep-100))
update.setdefault("data", {})
update["data"].setdefault("init_args", {})
update["data"]["init_args"].update(
    dict(
        global_stats=band_stats['t1']['ftw'],
        catalog=(
            f"{home_dir}/ftw-mappingafrica-integration/data/"\
            "ftw-catalog-long2.csv"
        )   
    )
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/1timepoint/{cfg_name}.yaml", 
           updates=update)